In [1]:
import numpy as np
import scipy.linalg as sla
import numpy.linalg as la
from scipy.special import expit as sigmoid
import scipy as sc
from statsmodels.tsa.api import VAR

In [2]:
varables = 3
observations = 1400
window = 2
#to use later 
times = 5

In [3]:
def qDag(w):
    return np.trace(sc.linalg.expm(np.multiply(w,w))) - w.shape[0]
def get_r(off):
    return lambda n: np.random.rand(n)+off

In [4]:
dagY = sc.sparse.random(varables, varables*window, density=0.15, data_rvs=get_r(.5)).A

dagX = np.eye(varables, varables)
while qDag(dagX) != 0:
    dagX = sc.sparse.random(varables, varables, density=0.25, data_rvs=get_r(.5)).A
    np.fill_diagonal(dagX, 0)

In [5]:
print(qDag(dagX))
print(dagX)
print(dagY)

0.0
[[0.         0.89071694 0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
[[0.         0.         0.         0.         0.         0.        ]
 [0.88865483 0.         0.         0.         1.00851057 0.        ]
 [0.         0.         0.         0.         0.         0.89025537]]


In [6]:
def rand(n):
    return 2*(np.random.rand(n)-.5)

Y_pre = rand(observations*varables*window).reshape(varables*window, observations)
print(dagY.shape)
print(Y_pre.shape)

(3, 6)
(6, 1400)


In [7]:
#X = XW + YA + Z

def randFlat(X):
    return rand(X.shape[0]*X.shape[1]).reshape(*X.shape)

def onesFlat(X):
    return np.ones(X.shape[0]*X.shape[1]).reshape(*X.shape)

def update(preY, preX, dagY, dagX, rand):
    Xy = np.matmul(dagY, preY)
    Xr = rand(preX)
    Xx = np.matmul(dagX, Xr)
    X = Xr + Xx + Xy
    
    return X

def step(preY, dagY, dagX, window, var, rand):
    preX = preY[:var]
    preY = preY[:window*var]
    return update(preY, preX, dagY, dagX, rand)

def makeSynthData(preY, times, obs, var, pad, rand=rand):
    window = preY.shape[0]//var
    
    for i in range(pad+window):
        newX = step(preY, dagY, dagX, window, var, rand)
        preY = np.append(newX, preY, axis=0)

    Y = preY[:window*var]

    for i in range(times - window):
        newX = step(Y, dagY, dagX, window, var, rand)
        Y = np.append(newX, Y, axis=0)
        
    return Y
    



#later
#W = np.random.rand(varables*observations).reshape(varables,observations)

In [8]:
Y_pre = np.ones(observations*varables*window).reshape(varables*window, observations)
XY = makeSynthData(Y_pre, times, observations, varables, 0, randFlat)


In [9]:
Y=XY[varables:window*varables+varables]
X=XY[:varables]

In [10]:
XY

array([[-0.34372468, -0.40113629, -0.22635544, ..., -0.66093134,
        -0.68677279,  0.12527215],
       [ 2.19619853,  2.42958787,  4.28345266, ...,  0.9245504 ,
        -0.56482533,  3.43890356],
       [ 0.88942179, -0.01945154,  0.38407955, ...,  0.24296432,
         0.42007838, -1.18955797],
       ...,
       [ 0.26495205,  0.37398413, -0.74142983, ..., -1.75008585,
         0.05731122,  0.91811098],
       [ 2.50706468,  1.2014802 ,  2.11233118, ...,  1.04057935,
         1.37772098,  2.71198897],
       [ 0.22775684,  1.85787558,  0.40766613, ...,  1.09041763,
         1.22146991, -0.04033202]])

In [11]:
C_init = np.random.rand(varables*varables*window + varables*varables)

In [12]:
def f_DYNOTEARS(X, Y, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(varables, varables*window)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

In [13]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.1,.1) , C_init, constraints=cons).x

In [14]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [15]:
A.round(1)

array([[-0. ,  0. ,  0. , -0. ,  0. ,  0. ],
       [ 0.7,  0. ,  0. , -0. ,  1. ,  0. ],
       [ 0. ,  0. , -0. , -0. ,  0. ,  0.8]])

In [16]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0.9, 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9]])

In [17]:
W.round(1)

array([[0. , 0. , 0. ],
       [0.2, 0. , 0. ],
       [0. , 0. , 0. ]])

In [18]:
dagX.round(1)

array([[0. , 0.9, 0. ],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

In [19]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0.9, 0. , 0. ],
       [0. , 0. , 0. ]])

In [20]:
def H1ldet(W):
    det = np.linalg.det(np.eye(W.shape[0]) - np.multiply(W,W))
    if (det <= 0):
        print(det)
    return -np.log(det)

def constr_DAGMA(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return H1ldet(w)

A_init = np.random.rand(varables*varables*window)

W_init = np.random.rand(varables*varables).reshape(varables, varables)
np.fill_diagonal(W_init, 0)
while H1ldet(W_init) is np.nan:
    W_init = np.random.rand(varables*varables).reshape(varables, varables)
    np.fill_diagonal(W, 0)

C_init = np.append(A_init, W_init.reshape(varables*varables), axis = 0)

cons = {'type':'eq', 'fun': constr_DAGMA}

#C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.01,.1) , C_init, constraints=cons).x

In [21]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [22]:
A[A<.3] = 0
A.round()

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [23]:
dagY.round()

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [24]:
W.round(1)

array([[0. , 0. , 0. ],
       [0.2, 0. , 0. ],
       [0. , 0. , 0. ]])

In [25]:
dagX.round(1)

array([[0. , 0.9, 0. ],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

In [26]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0.9, 0. , 0. ],
       [0. , 0. , 0. ]])

In [27]:
C_init = np.random.rand(varables*varables*window + varables*varables)
H1ldet(C_init.T)

-7.040788595097462


/var/folders/z_/p0tz320d6j13q0rkv_f35brm0000gn/T/ipykernel_19124/3164180601.py:5: RuntimeWarning: invalid value encountered in log
  return -np.log(det)


nan

In [28]:
def metrics(A,dagY, W, dagX):
    A = np.absolute(A) > 0.3
    dagY = np.absolute(dagY) > .5
    W = np.absolute(W) > 0.3
    dagX = np.absolute(dagX) > .5
    
    preWrong = A != dagY
    wWrong = W != dagX
    #assumes cant be both wrong and inv
    wInverse = W.T & dagX
    
    if ((W + W.T)==2).any():
        print(W)
        print(W.T)
        raise "error"
    
    #print(A)
    #print(dagY)
    #print(W)
    #print(dagX)
    #print()
    #print(preWrong)
    #print(wWrong)
    #print(wInverse)
    
    return {
        "preWrong": sum(sum(preWrong)),
        "wWrong": sum(sum(wWrong)),
        "wInverse": sum(sum(wInverse))
    }
    

In [29]:
metrics(A,dagY, W, dagX)

{'preWrong': 0, 'wWrong': 1, 'wInverse': 0}

In [30]:
A.round()

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [31]:
dagY.round()

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [32]:
np.matmul(np.eye(varables, varables) - W, X).round(1)

array([[-0.3, -0.4, -0.2, ..., -0.7, -0.7,  0.1],
       [ 2.3,  2.5,  4.3, ...,  1.1, -0.4,  3.4],
       [ 0.9, -0. ,  0.4, ...,  0.2,  0.4, -1.2]])

In [33]:
def transformTimeData(data, window, times):
    variables = data.shape[0]//times
    print(data.shape[0]/times)
    
    snapshots = times - window - 1
    
    ret = []
    
    for i in range(snapshots):
        slic = data[i*variables:i*variables+(window+1)*variables]
        ret.append( slic)
    
    return np.hstack(ret)
    
    

In [34]:
XY_stack = transformTimeData(XY, window, times)

3.0


In [35]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]


In [36]:
print(XY.shape)
print(XY_stack.shape)
print(Y_stack.shape)
print(X_stack.shape)

(15, 1400)
(9, 2800)
(6, 2800)
(3, 2800)


In [37]:
XY.shape

(15, 1400)

In [38]:
XY_stack

array([[-0.34372468, -0.40113629, -0.22635544, ..., -0.1861515 ,
        -0.67512727,  0.75099588],
       [ 2.19619853,  2.42958787,  4.28345266, ..., -1.25204623,
         0.85430509,  1.46265011],
       [ 0.88942179, -0.01945154,  0.38407955, ...,  0.92345613,
         1.29140859,  1.86370553],
       ...,
       [-1.07596974,  0.74463185,  1.15278123, ..., -0.71351337,
        -1.09013413, -0.92316426],
       [ 2.82273775,  1.87635302,  3.30889025, ..., -0.81495132,
         0.81669791,  1.4320431 ],
       [ 0.46669791,  0.88589757, -0.22659974, ...,  0.52057461,
         0.54160716,  1.1469347 ]])

In [39]:
X_stack

array([[-0.34372468, -0.40113629, -0.22635544, ..., -0.1861515 ,
        -0.67512727,  0.75099588],
       [ 2.19619853,  2.42958787,  4.28345266, ..., -1.25204623,
         0.85430509,  1.46265011],
       [ 0.88942179, -0.01945154,  0.38407955, ...,  0.92345613,
         1.29140859,  1.86370553]])

In [40]:
Y_stack

array([[ 0.04735874,  0.04764766,  0.15486903, ...,  0.51906431,
         0.62995604,  0.02235007],
       [ 1.95258704,  3.13947579,  1.31961228, ...,  0.93492239,
         1.02517199,  2.31976387],
       [-0.39825605,  0.9069102 ,  0.75125138, ...,  0.22892889,
         1.34718742, -0.42919074],
       [-1.07596974,  0.74463185,  1.15278123, ..., -0.71351337,
        -1.09013413, -0.92316426],
       [ 2.82273775,  1.87635302,  3.30889025, ..., -0.81495132,
         0.81669791,  1.4320431 ],
       [ 0.46669791,  0.88589757, -0.22659974, ...,  0.52057461,
         0.54160716,  1.1469347 ]])

In [41]:
cons = {'type':'eq', 'fun': constr_DYNOTEARS}
C = sc.optimize.minimize(f_DYNOTEARS(X_stack,Y_stack,.01,.1) , C_init, constraints=cons).x

In [42]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [43]:
abs(A.round(1))

array([[0.3, 0. , 0. , 0. , 0.3, 0. ],
       [0.9, 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9]])

In [44]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0.9, 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9]])

In [45]:
W.round(2)

array([[ 0.  ,  0.33, -0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [-0.  ,  0.  ,  0.  ]])

In [46]:
dagX.round(1)

array([[0. , 0.9, 0. ],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

In [47]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0.9, 0. , 0. ],
       [0. , 0. , 0. ]])

In [48]:
#tensor representation for timeseries
dagYtesnor = np.transpose(np.array([dagY[:,i*varables:(i+1)*varables] for i in range(window)]),(0,2,1))

dagXtesnor = dagX
print(dagYtesnor.round().shape)

XYtesnor = XY.reshape(times, varables, observations)
print(XYtesnor.shape)

(2, 3, 3)
(5, 3, 1400)


In [49]:
XY_stack

array([[-0.34372468, -0.40113629, -0.22635544, ..., -0.1861515 ,
        -0.67512727,  0.75099588],
       [ 2.19619853,  2.42958787,  4.28345266, ..., -1.25204623,
         0.85430509,  1.46265011],
       [ 0.88942179, -0.01945154,  0.38407955, ...,  0.92345613,
         1.29140859,  1.86370553],
       ...,
       [-1.07596974,  0.74463185,  1.15278123, ..., -0.71351337,
        -1.09013413, -0.92316426],
       [ 2.82273775,  1.87635302,  3.30889025, ..., -0.81495132,
         0.81669791,  1.4320431 ],
       [ 0.46669791,  0.88589757, -0.22659974, ...,  0.52057461,
         0.54160716,  1.1469347 ]])

In [50]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]

In [51]:
Y_stack

array([[ 0.04735874,  0.04764766,  0.15486903, ...,  0.51906431,
         0.62995604,  0.02235007],
       [ 1.95258704,  3.13947579,  1.31961228, ...,  0.93492239,
         1.02517199,  2.31976387],
       [-0.39825605,  0.9069102 ,  0.75125138, ...,  0.22892889,
         1.34718742, -0.42919074],
       [-1.07596974,  0.74463185,  1.15278123, ..., -0.71351337,
        -1.09013413, -0.92316426],
       [ 2.82273775,  1.87635302,  3.30889025, ..., -0.81495132,
         0.81669791,  1.4320431 ],
       [ 0.46669791,  0.88589757, -0.22659974, ...,  0.52057461,
         0.54160716,  1.1469347 ]])

In [52]:
X_stack

array([[-0.34372468, -0.40113629, -0.22635544, ..., -0.1861515 ,
        -0.67512727,  0.75099588],
       [ 2.19619853,  2.42958787,  4.28345266, ..., -1.25204623,
         0.85430509,  1.46265011],
       [ 0.88942179, -0.01945154,  0.38407955, ...,  0.92345613,
         1.29140859,  1.86370553]])

In [53]:
def diff(X, w, a):
    X_check = X[:X.shape[0]-a.shape[0]]
    
    Xw = np.matmul(w, X_check)
    #print(a.shape)
    #print(X.shape[0]-window+1)
    Xa = np.array([np.tensordot(a, X[i:i+window], axes=((0,1),(0,1)))
                    for i in range(1, X.shape[0]-window+1)])
    #print(X_check.shape)
    #print(Xw.shape)
    #print(Xa.shape)
    
    return X_check - Xw - Xa

def f_DYNOTEARS_TENSOR(X, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(window, varables, varables)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(diff(X, w, a)).sum().sum().sum()/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

def ll(X, Y, w, a):
    return np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)

In [54]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]

In [55]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS_TENSOR(XYtesnor,.05,.05) , C_init, constraints=cons).x

In [56]:
A = C[:varables*window*varables].reshape(window, varables, varables)
W = C[varables*window*varables:].reshape(varables, varables)
print(ll(X_stack, Y_stack, dagX, dagY))

#print(ll(XYtesnor, W, A))

print(np.square(diff(XYtesnor, dagX, dagYtesnor)).sum().sum().sum()/np.prod(XYtesnor.shape))

print(np.square(diff(XYtesnor, W, A)).sum().sum().sum()/np.prod(XYtesnor.shape))

1.3303816179645873
0.8390758387672083
0.23629601371727035


In [57]:
A.shape

(2, 3, 3)

In [58]:
W.shape

(3, 3)

In [59]:
A.round(1)

array([[[ 0. ,  0.7,  0. ],
        [-0. ,  0. ,  0. ],
        [-0. ,  0. , -0. ]],

       [[-0. , -0. ,  0. ],
        [ 0. ,  1. ,  0. ],
        [-0. ,  0. ,  0.8]]])

In [60]:
dagYtesnor.round(1)

array([[[0. , 0.9, 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 1. , 0. ],
        [0. , 0. , 0.9]]])

In [61]:
XYtesnor[1:1+window].round()

array([[[ 0.,  0.,  0., ..., -0., -1.,  1.],
        [ 2.,  3.,  1., ..., -1.,  1.,  1.],
        [-0.,  1.,  1., ...,  1.,  1.,  2.]],

       [[-1.,  1.,  1., ...,  1.,  1.,  0.],
        [ 3.,  2.,  3., ...,  1.,  1.,  2.],
        [ 0.,  1., -0., ...,  0.,  1., -0.]]])

In [62]:
dagYtesnor.round(1)

array([[[0. , 0.9, 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 1. , 0. ],
        [0. , 0. , 0.9]]])

In [63]:
np.tensordot(A, XYtesnor[1:1+window], axes=((0,1),(0,1))).round(1)

array([[ 0. , -0. ,  0. , ...,  0. , -0. ,  0. ],
       [ 2.8,  1.9,  3.3, ...,  0.8,  0.5,  2.8],
       [ 0.4,  0.7, -0.2, ...,  0.2,  1.1, -0.3]])

In [64]:
XYtesnor[0].round()
XYtesnor[1:1+window].round()

array([[[ 0.,  0.,  0., ..., -0., -1.,  1.],
        [ 2.,  3.,  1., ..., -1.,  1.,  1.],
        [-0.,  1.,  1., ...,  1.,  1.,  2.]],

       [[-1.,  1.,  1., ...,  1.,  1.,  0.],
        [ 3.,  2.,  3., ...,  1.,  1.,  2.],
        [ 0.,  1., -0., ...,  0.,  1., -0.]]])

In [65]:
#np.array([dagY[:,i*varables:(i+1)*varables] for i in range(window)]).round(1)

In [66]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0.9, 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9]])

In [67]:
Y_stack.round()

array([[ 0.,  0.,  0., ...,  1.,  1.,  0.],
       [ 2.,  3.,  1., ...,  1.,  1.,  2.],
       [-0.,  1.,  1., ...,  0.,  1., -0.],
       [-1.,  1.,  1., ..., -1., -1., -1.],
       [ 3.,  2.,  3., ..., -1.,  1.,  1.],
       [ 0.,  1., -0., ...,  1.,  1.,  1.]])

In [68]:
np.matmul(dagY, Y_stack).round()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  2.,  3., ..., -0.,  1.,  1.],
       [ 0.,  1., -0., ...,  0.,  0.,  1.]])

In [69]:
X_stack.round()

array([[-0., -0., -0., ..., -0., -1.,  1.],
       [ 2.,  2.,  4., ..., -1.,  1.,  1.],
       [ 1., -0.,  0., ...,  1.,  1.,  2.]])

In [70]:
dagX.round(1)

array([[0. , 0.9, 0. ],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

In [71]:
W.round(1)

array([[ 0. ,  0. ,  0. ],
       [ 0.3,  0. ,  0. ],
       [-0. ,  0. ,  0. ]])

In [90]:
def get_sparse(X):
    ret = []
    for i in range(len(X)):
        for j in range(len(X[i])):
            val = X[i,j]
            if not np.isnan(val):
                print(val)
                ret.append((i, j, val))
    return ret

constraint_matrix_X = np.array(
[
    [np.nan, np.nan, np.nan],
    [np.nan, np.nan, np.nan],
    [0, 1.1, np.nan],
])
X_set = np.logical_not(np.isnan(constraint_matrix_X)).sum().sum()
print(X_set)
X_sparce = get_sparse(constraint_matrix_X)

            
constraint_matrix_Y = np.array(
[
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    [np.nan, np.nan, np.nan, 6, np.nan, np.nan],
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
])
Y_set = np.logical_not(np.isnan(constraint_matrix_Y)).sum().sum()
print(Y_set)
Y_sparce = get_sparse(constraint_matrix_Y)


#C_init = np.random.rand(varables*varables*window + varables*varables)
#not create no C_init with XYset less variables and instread enforce in loss func
#to avoid unnessasary copying

2
0.0
1.1
1
6.0


In [91]:
def enforce_cons(X, cons):
    for (i, j, val) in cons:
        X[i][j] = val

def f_DYNOTEARS_CONSTRAINT(X, Y, lA, lB, X_cons, Y_cons):
    def aux(C):
        w = C[varables*window*varables:].reshape(varables, varables)
        enforce_cons(w, X_cons)
        a = C[:varables*window*varables].reshape(varables, varables*window)
        enforce_cons(a, Y_cons)
    
        loss = np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

In [92]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS_CONSTRAINT(X,Y,.1,.1,X_sparce, Y_sparce) , C_init, constraints=cons).x

In [93]:

W = C[varables*window*varables:].reshape(varables, varables)
enforce_cons(W, X_sparce)
A = C[:varables*window*varables].reshape(varables, varables*window)
enforce_cons(A, Y_sparce)
np.fill_diagonal(W, 0)

In [97]:
enforce_cons(A, Y_sparce)
A.round(1)

array([[ 0. , -0. , -0. ,  0. ,  0. ,  0. ],
       [ 1.7, -1.9,  0.5,  6. ,  1.5,  0.3],
       [-0.7, -0. , -0. ,  0. , -1. ,  0.6]])

In [98]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0.9, 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9]])

In [99]:
W.round(1)

array([[0. , 0. , 0. ],
       [0.3, 0. , 0. ],
       [0. , 1.1, 0. ]])

In [100]:
dagX.round(1)

array([[0. , 0.9, 0. ],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])